In [82]:
import random
import json
import numpy as np

In [83]:
class Network(object):

    def __init__(self, sizes):
      self.num_layers = len(sizes)
      self.sizes = sizes
      self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
      self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
      for b, w in zip(self.biases, self.weights):
        a = sigmoid(np.dot(w, a)+b)
      return a

    def stochastic_gradient_descent(self, training_data, epochs, mini_batch_size, learning_rate, test_data=None, export_params=False):
      training_data = list(training_data)
      n = len(training_data)

      if test_data:
        test_data = list(test_data)
        n_test = len(test_data)

      for j in range(epochs):
        random.shuffle(training_data)
        mini_batches = [
          training_data[k:k+mini_batch_size]
          for k in range(0, n, mini_batch_size)]
        for mini_batch in mini_batches:
          self.update_mini_batch(mini_batch, learning_rate)
        if j % 3 == 0:
          if test_data:
            print("Epoch {} : {}% acuracy".format(j,self.evaluate(test_data) * 100 / n_test))
          else:
            print("Epoch {} complete".format(j))

      if export_params:
        with open('data.json', 'w') as outfile:
          outfile.write('data = ')
          json.dump({
            "weights": self.weights,
            "biases": self.biases
          }, outfile, cls=NumpyEncoder)

    def update_mini_batch(self, mini_batch, learning_rate):
      nabla_b = [np.zeros(b.shape) for b in self.biases]
      nabla_w = [np.zeros(w.shape) for w in self.weights]
      for x, y in mini_batch:
        delta_nabla_b, delta_nabla_w = self.backprop(x, y)
        nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
        nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
      
      self.weights = [w-(learning_rate/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)]
      self.biases = [b-(learning_rate/len(mini_batch))*nb for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
      nabla_b = [np.zeros(b.shape) for b in self.biases]
      nabla_w = [np.zeros(w.shape) for w in self.weights]
      activation = x
      activations = [x]
      zs = []
      for b, w in zip(self.biases, self.weights):
        z = np.dot(w, activation)+b
        zs.append(z)
        activation = sigmoid(z)
        activations.append(activation)
      delta = self.cost_derivative(activations[-1], y) * \
        sigmoid_prime(zs[-1])
      nabla_b[-1] = delta
      nabla_w[-1] = np.dot(delta, activations[-2].transpose())
      for l in range(2, self.num_layers):
        z = zs[-l]
        sp = sigmoid_prime(z)
        delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
        nabla_b[-l] = delta
        nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
      return (nabla_b, nabla_w)

    def evaluate(self, test_data):
      test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
      
      return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
      return (output_activations-y)

def sigmoid(z):
  return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
  return sigmoid(z)*(1-sigmoid(z))

class NumpyEncoder(json.JSONEncoder):
  def default(self, obj):
    if isinstance(obj, np.ndarray):
      return obj.tolist()
    return json.JSONEncoder.default(self, obj)

In [84]:
import mnist_loader
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()

In [85]:
net = Network([784, 25, 25, 10])
net.stochastic_gradient_descent(training_data, 30, 10, 3.0, test_data=test_data, export_params=True)

Epoch 0 : 90.4% acuracy
Epoch 3 : 92.75% acuracy
Epoch 6 : 93.74% acuracy
Epoch 9 : 94.12% acuracy
Epoch 12 : 94.22% acuracy
Epoch 15 : 94.23% acuracy
Epoch 18 : 94.46% acuracy
Epoch 21 : 94.37% acuracy
Epoch 24 : 94.68% acuracy
Epoch 27 : 94.64% acuracy
